In [4]:
import datetime as dt

import pandas as pd

from openbb_terminal.sdk import openbb
import quantstats as qs
import backtrader as bt

In [5]:
def openbb_data_to_bt_data(symbol, start_date, end_date):
    
    df = openbb.stocks.load(symbol, start_date=start_date, end_date=end_date)
    
    

    fn = f"{symbol.lower()}.csv"
    df.to_csv(fn)
    
    return bt.feeds.YahooFinanceCSVData(
        dataname=fn,
        fromdate=dt.datetime.strptime(start_date, '%Y-%m-%d'),
        todate=dt.datetime.strptime(end_date, '%Y-%m-%d')
    )

In [11]:
# Define the Bollinger Bands strategy
class BollingerBandsStrategy(bt.Strategy):
    params = (('period', 20), ('devfactor', 2.0),)

    def __init__(self):
        self.boll = bt.indicators.BollingerBands(self.data.close, period=self.p.period, devfactor=self.p.devfactor)

    def next(self):
        if self.position.size:
            if self.data.close[0] >= self.boll.lines.top:
                self.sell()
            elif self.data.close[0] <= self.boll.lines.bot:
                self.buy()
        elif self.data.close[0] <= self.boll.lines.mid:
            self.buy()
        elif self.data.close[0] >= self.boll.lines.mid:
            self.sell()

In [13]:
data = openbb_data_to_bt_data(
    "RTX", 
    start_date="2021-01-01",
    end_date="2023-06-29"
)

cerebro = bt.Cerebro(stdstats=False)

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)

cerebro.addstrategy(BollingerBandsStrategy)

cerebro.addobserver(bt.observers.Value)

cerebro.addanalyzer(
    bt.analyzers.Returns, _name="returns"
)
cerebro.addanalyzer(
    bt.analyzers.TimeReturn, _name="time_return"
)

backtest_result = cerebro.run()

Loading Daily data for RTX with starting period 2021-01-04.

In [15]:
# Get the strategy returns as a dictionary
returns_dict = backtest_result[0].analyzers.time_return.get_analysis()

# Convert the dictionary to a DataFrame
returns_df = (
    pd.DataFrame(
        list(returns_dict.items()),
        columns = ["date", "return"]
    )
    .set_index("date")
)

In [16]:
bench = openbb.stocks.load(
    "RTX",
    start_date="2021-01-01",
    end_date="2023-06-29"
)["Adj Close"]

qs.reports.metrics(
    returns_df,
    benchmark=bench,
    mode="full"
)

Loading Daily data for RTX with starting period 2021-01-04.

                           Strategy    Benchmark
-------------------------  ----------  -----------
Start Period               2021-01-04  2021-01-04
End Period                 2023-06-28  2023-06-28
Risk-Free Rate             0.0%        0.0%
Time in Market             97.0%       100.0%

Cumulative Return          -2.95%      48.58%
CAGR﹪                     -1.2%       17.31%

Sharpe                     0.08        0.8
Prob. Sharpe Ratio         54.77%      89.62%
Smart Sharpe               0.07        0.77
Sortino                    0.11        1.2
Smart Sortino              0.11        1.16
Sortino/√2                 0.08        0.85
Smart Sortino/√2           0.08        0.82
Omega                      1.01        1.01

Max Drawdown               -26.52%     -21.92%
Longest DD Days            449         361
Volatility (ann.)          25.01%      23.43%
R^2                        0.72        0.72
Information Ratio          -0.02       -0.02
Calmar                     -0.05       